# Coming Soon...
- Display control system diagram
- Explain input-output ports
- Sending commands programmatically (controller class with inputs and outpus)
- Challenges: 
    - Send end-effector to specfic point
    - Move in a circle
    - Pick up block at known location